In [4]:
import cv2
import numpy as np

yolo = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []

with open("coco.names", "r") as file:
    classes = [line.strip() for line in file.readlines()]
layer_names = yolo.getLayerNames()
output_layers = [layer_names[i - 1] for i in yolo.getUnconnectedOutLayers()]
colorRed = (0, 0, 255)
colorGreen = (0, 255, 0)

# Open the camera
cap = cv2.VideoCapture(0)  # Use 0 for the default camera, or change to the appropriate camera index

# Get the screen dimensions
screen_width = int(cap.get(3))
screen_height = int(cap.get(4))

# Calculate the padding
padding = 20
padded_width = screen_width - 2 * padding
padded_height = screen_height - 2 * padding

while True:
    ret, img = cap.read()  # Read a frame from the camera
    height, width, channels = img.shape

    # Resize the frame to match the padded dimensions
    img = cv2.resize(img, (padded_width, padded_height))

    # Create a black background with padding
    padded_frame = np.zeros((screen_height, screen_width, 3), dtype=np.uint8)
    padded_frame[padding:padded_height + padding, padding:padded_width + padding] = img

    # Detecting objects
    blob = cv2.dnn.blobFromImage(padded_frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    yolo.setInput(blob)
    outputs = yolo.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * padded_width)
                center_y = int(detection[1] * padded_height)
                w = int(detection[2] * padded_width)
                h = int(detection[3] * padded_height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            # Adjust coordinates for the padding
            x += padding
            y += padding
            cv2.rectangle(padded_frame, (x, y), (x + w, y + h), colorGreen, 3)
            cv2.putText(padded_frame, label, (x, y + 10), cv2.FONT_HERSHEY_PLAIN, 1, colorRed, 2)

    cv2.namedWindow("Camera Feed", cv2.WND_PROP_FULLSCREEN)
    cv2.setWindowProperty("Camera Feed", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    cv2.imshow("Camera Feed", padded_frame)

    if cv2.waitKey(1) == ord("q"):  # Press 'q' to exit the live feed
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
